In [31]:
import numpy as np

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

from transformers import AutoTokenizer, DataCollatorWithPadding

In [32]:
# !pip install datasets

In [33]:
raw_datasets = load_from_disk("data/finetune_data/")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

Map: 100%|██████████| 5791/5791 [00:00<00:00, 16099.26 examples/s]


In [34]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 808698
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 346585
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5791
    })
})

In [35]:
raw_datasets['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 808698
})

In [36]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [37]:
# !pip install transformers[torch]
# !pip install accelerate -U

In [38]:
# !pip install accelerate==0.24.0

In [39]:
import accelerate
print(accelerate.__version__)

0.24.0


In [40]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to match evaluation strategy
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
trainer.train()

  0%|          | 0/361030 [00:17<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(1725, 2) (1725,)


In [ ]:
predictions

PredictionOutput(predictions=array([[-4.521311 ,  3.7222323],
       [-3.4110165,  2.9014902],
       [-4.623328 ,  3.8644013],
       ...,
       [-4.612073 ,  3.8569236],
       [-4.514099 ,  3.6752856],
       [-4.265427 ,  3.4012394]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.8783374428749084, 'test_accuracy': 0.8417391304347827, 'test_f1': 0.8847615027437737, 'test_runtime': 10.9624, 'test_samples_per_second': 157.356, 'test_steps_per_second': 1.277})

In [ ]:
import numpy as np
from scipy.special import softmax

# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[2.6288169e-04 9.9973708e-01]
 [1.8101990e-03 9.9818975e-01]
 [2.0593787e-04 9.9979407e-01]
 ...
 [2.0983144e-04 9.9979013e-01]
 [2.7750764e-04 9.9972242e-01]
 [4.6795682e-04 9.9953210e-01]]
Predicted Labels:
 [1 1 1 ... 1 1 1]
